In [2]:
import csv
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import cv2 as cv

In [3]:
from computational_inpainting import inpaint

In [4]:
DATA_PATH = '/media/disk2/amaltsev/car_lp_generator'

In [5]:
old_h = 85  # 115 mm
old_w = 385  # 520 mm
new_h = 126  # 170 mm
new_w = 217  # 290 mm

In [6]:
template = cv.imread(os.path.join(DATA_PATH, 'templates', 'template.png'))
template_new_mask = cv.imread(os.path.join(DATA_PATH, 'templates', 'template_new_mask.png'), cv.IMREAD_GRAYSCALE)

In [7]:
alph_map = {
    'а': 'A',
    'в': 'B',
    'е': 'E',
    'к': 'K',
    'м': 'M',
    'н': 'H',
    'о': 'O',
    'р': 'P',
    'с': 'C',
    'т': 'T',
    'у': 'Y',
    'х': 'X'
}

def text_format(text):
    if len(text) == 8:
        text = text[:6] + '_' + text[-2:]
    new_text = ''
    for i in range(len(text)):
        if text[i] in alph_map:
            new_text += alph_map[text[i]]
        else:
            new_text += text[i]
            
    return new_text

In [8]:
sym_imgs = {}
for sym in '0123456789ABEKMHOPCTYX':
    img = cv.imread(os.path.join(DATA_PATH, 'templates', sym + '.png'))
    sym_imgs[sym] = img

In [9]:
def gen_base(template, text):
    sign = template
    sizes = [[10, 36, 45, 31],
             [10, 89, 45, 28],
             [10, 122, 45, 28],
             [10, 155, 45, 28],
             [70, 23, 45, 31],
             [70, 63, 45, 31],
             [69, 128, 35, 17],
             [69, 152, 35, 22],
             [69, 178, 35, 22]]

    for i in range(len(text)):
        sym = text[i]
        if sym == '_':
            continue
        img = sym_imgs[text[i]]
        img = cv.resize(img, (int(img.shape[1] * sizes[i][2] / img.shape[0]), sizes[i][2]))

        # print(img.shape)
        # cv.imshow('symbol', img)
        # cv.waitKey(0)

        sign[sizes[i][0]:sizes[i][0] + img.shape[0], sizes[i][1]:sizes[i][1] + img.shape[1]] = img

    return sign

In [10]:
def crop_center(image):
    """Returns a cropped square image."""
    shape = image.shape
    new_shape = min(shape[1], shape[2])
    offset_y = max(shape[1] - shape[2], 0) // 2
    offset_x = max(shape[2] - shape[1], 0) // 2
    image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
    return image

# @functools.lru_cache(maxsize=None)
# def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
def load_image(img, image_size=(256, 256), preserve_aspect_ratio=True):
    """Loads and preprocesses images."""
    # Cache image file locally.
    #   image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
    # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
    #   img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
    img = img.astype(np.float32)[np.newaxis, ...]
    if img.max() > 1.0:
        img = img / 255.
    if len(img.shape) == 3:
        img = tf.stack([img, img, img], axis=-1)
        img = crop_center(img)
        img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
    return img

def show_n(images, titles=('',)):
    n = len(images)
    image_sizes = [image.shape[1] for image in images]
    w = (image_sizes[0] * 6) // 320
    plt.figure(figsize=(w  * n, w))
    gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
    for i in range(n):
        plt.subplot(gs[i])
        plt.imshow(images[i][0], aspect='equal')
        plt.axis('off')
        plt.title(titles[i] if len(titles) > i else '')
    plt.show()
    
def tensor_to_img(tensor):
    image_size = tensor.shape[1]

In [11]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

In [32]:
def transfer_style(content, style):
    content_image = load_image(content)
    style_image = load_image(style)

    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')

    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0]

    stylized_image = stylized_image.numpy()[0]
    stylized_image *= 255
    stylized_image = stylized_image.astype(np.uint8)
    cv.imwrite(os.path.join(DATA_PATH, 'transferred', filename), stylized_image)

    stylized_image = tf.image.resize(stylized_image, (126, 217))
    
    return np.array(stylized_image)

In [33]:
dilation_kernel_size = 3 * 2 + 1

In [34]:
import time

In [36]:
start_time = time.time()

with open(os.path.join(DATA_PATH, 'abbyy_train.csv'), newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    i = 0
    for row in reader:
#         try:
        filename = row['File']
        coords = [int(x) for x in row['Coords'].split(',')]
        corners = [coords[0:2], coords[2:4], coords[4:6], coords[6:8]]

        corners.sort(key=lambda x: x[0])
        left = sorted(corners[:2], key=lambda x: x[1])
        right = sorted(corners[2:], key=lambda x: x[1])
        corners = left + right

        corners_dst = [[old_w // 2, 2 * old_h], [old_w // 2, 3 * old_h],
                       [3 * old_w // 2, 2 * old_h], [3 * old_w // 2, 3 * old_h]]
        M = cv.getPerspectiveTransform(np.array(corners, dtype='float32'),
                                       np.array(corners_dst, dtype='float32'))

        img = cv.imread(os.path.join(DATA_PATH, 'frames_train', filename))
        warped = cv.warpPerspective(img, M, (old_w * 2, old_h * 5), borderMode=cv.BORDER_CONSTANT)

#         style_src_sq = warped[(5 * old_h - old_w) // 2: (5 * old_h + old_w) // 2, old_w // 2: 3 * old_w // 2]
        style_src = warped[2 * old_h: 3 * old_h, old_w // 2: 3 * old_w // 2]
        style_src_sq = cv.resize(style_src, (256, 256))
        
        cv.imwrite(os.path.join(DATA_PATH, 'style_src_sq', filename), style_src_sq)
        
#         style_src_sq = cv.GaussianBlur(style_src_sq, (7,7), 0)
        style_src_sq_white = cv.dilate(style_src_sq, cv.getStructuringElement(cv.MORPH_RECT, (dilation_kernel_size, dilation_kernel_size)))
        cv.imwrite(os.path.join(DATA_PATH, 'style_src_white.jpg'), style_src_sq_white)
        
#         cv.imwrite(os.path.join(DATA_PATH, 'style_src_sq', filename), style_src_sq)

        inpainted = inpaint(warped)
        cv.imwrite(os.path.join(DATA_PATH, 'inpainted', filename), inpainted)

        text = text_format(row['Number'])
        new_plate = np.zeros(warped.shape, dtype=np.uint8)
#         print(new_plate.shape)
        base = gen_base(template.copy(), text)
        style_dst_sq = cv.resize(base, (256, 256))
        new_plate[(warped.shape[0] - template_new_mask.shape[0]) // 2 + 15: (warped.shape[0] + template_new_mask.shape[0]) // 2 + 15,
                  (warped.shape[1] - template_new_mask.shape[1]) // 2: (warped.shape[1] + template_new_mask.shape[1]) // 2, :
                 ] = base
#         cv.imwrite(os.path.join(DATA_PATH, 'new_plate.jpg'), new_plate)

        mask = np.zeros(warped.shape[:2], dtype=np.uint8)
        mask[(warped.shape[0] - template_new_mask.shape[0]) // 2 + 15: (warped.shape[0] + template_new_mask.shape[0]) // 2 + 15,
             (warped.shape[1] - template_new_mask.shape[1]) // 2: (warped.shape[1] + template_new_mask.shape[1]) // 2
            ] = template_new_mask
#         cv.imwrite(os.path.join(DATA_PATH, 'new_plate.jpg'), mask)

#         inpainted[mask==255] = new_plate[mask==255]
#         cv.imwrite(os.path.join(DATA_PATH, 'inpainted_rendered', filename), inpainted)

#         style_dst_sq = inpainted[(5 * old_h - old_w) // 2: (5 * old_h + old_w) // 2, old_w // 2: 3 * old_w // 2]

        stylized_image = transfer_style(style_dst_sq, style_src_sq)
        stylized_white = transfer_style(style_dst_sq, style_src_sq_white)
        
        base_gray = cv.cvtColor(base, cv.COLOR_BGR2GRAY).astype(int)
        
        stylized_image[base_gray>230] = stylized_white[base_gray>230]

        stylized_ext = np.zeros(warped.shape, dtype=np.uint8)
        stylized_ext[(warped.shape[0] - stylized_image.shape[0]) // 2 + 15: (warped.shape[0] + stylized_image.shape[0]) // 2 + 15,
                     (warped.shape[1] - stylized_image.shape[1]) // 2: (warped.shape[1] + stylized_image.shape[1]) // 2
                    ] = stylized_image
        inpainted[mask==255] = stylized_ext[mask==255]
        
        unwarped = cv.warpPerspective(inpainted, M, (img.shape[1], img.shape[0]), img,
                                      flags=cv.WARP_INVERSE_MAP, borderMode=cv.BORDER_TRANSPARENT)    

#             cv.imwrite(os.path.join(DATA_PATH, 'inpainted_rendered', filename), inpainted)
        cv.imwrite(os.path.join(DATA_PATH, 'final', filename), unwarped)
        
        corners_new = [
            [(warped.shape[1] - template_new_mask.shape[1]) // 2, (warped.shape[0] - template_new_mask.shape[0]) // 2 + 15],
            [(warped.shape[1] + template_new_mask.shape[1]) // 2, (warped.shape[0] - template_new_mask.shape[0]) // 2 + 15],
            [(warped.shape[1] - template_new_mask.shape[1]) // 2, (warped.shape[0] + template_new_mask.shape[0]) // 2 + 15],
            [(warped.shape[1] + template_new_mask.shape[1]) // 2, (warped.shape[0] + template_new_mask.shape[0]) // 2 + 15]
        ]
        corners_unwarped = cv.perspectiveTransform(np.array(corners_new).reshape(-1,1,2).astype(np.float32), np.linalg.inv(M))
        
#         print(corners_new)
#         print(np.array(corners_new).reshape(-1,1,2))
        
        corners_unwarped = corners_unwarped.reshape(-1, 2)
#         print(corners_unwarped)
#         print(corners)
        
        unwarped_crop = unwarped[int(sorted(corners_unwarped, key=lambda x: x[1])[0][1]): int(sorted(corners_unwarped, key=lambda x: x[1])[-1][1]), 
                                 int(sorted(corners_unwarped, key=lambda x: x[0])[0][0]): int(sorted(corners_unwarped, key=lambda x: x[0])[-1][0])]
        cv.imwrite(os.path.join(DATA_PATH, 'final_crop', filename), unwarped_crop)
    
        if i % 100 == 0:
            print(i)
            print(time.time() - start_time)
        i += 1

#         except Exception as e:
#             print(row, e)

0
0.4540238380432129
100
46.27618956565857
200
92.15995812416077
300
138.38766264915466
400
184.7697033882141
500
231.4110198020935
600
277.39201188087463
700
323.7458381652832
800
369.9699227809906
900
416.32864570617676
1000
462.96546244621277
1100
509.40345191955566
1200
556.0872905254364
1300
602.3652102947235
1400
648.7673273086548
1500
695.4214689731598
1600
741.6491007804871
1700
788.0193583965302
1800
834.3526730537415
1900
881.1606950759888
2000
927.2942111492157
2100
972.9677696228027
2200
1018.9736747741699
2300
1064.883013010025
2400
1110.8437781333923
2500
1157.1141963005066
2600
1203.0965616703033
2700
1249.7145237922668
2800
1295.8492217063904
2900
1342.1216578483582
3000
1388.6176857948303
3100
1434.9764890670776
3200
1480.781972169876
3300
1526.5732550621033
3400
1572.6612079143524
3500
1619.0503027439117
3600
1665.468210220337
3700
1711.4783511161804
3800
1757.553943157196
3900
1803.8745062351227
4000
1849.81134557724
4100
1895.7941732406616
4200
1941.897586107254
430